In [128]:
import pandas as pd

df = pd.read_csv('/root/KRX_LLM/data/krx_sample_mcqa_data4.csv')
def is_numeric(val):
    try:
        int(val)
        return True
    except ValueError:
        return False

print(len(df))
df = df[df['answer'].apply(is_numeric)]
df = df.reset_index(drop=True)

df['question'] = df['question'].map(lambda x : x)
print(len(df))

100
98


In [129]:
df.head(10)

,question,answer
0,블랙-숄즈(Black-Scholes) 모델을 사용하여 감마(Gamma)를 계산할 때...,3
1,"다음 수식으로 정의된 파생상품의 지급금 가격을 계산하고, 이 가격이 주어진 유럽형 ...",0
2,스팟 가격과 옵션 내재 변동성 간의 상관관계인 $\rho$의 중요성에 대해 설명할 ...,5
3,"GARCH(1,1) 모델을 사용할 때, 어떤 이유로 해당 모델을 선택하는 것이 잘못...",1
4,금융 시장에서 월요일 효과와 1월 효과가 나타나는 원인에 대해 다음 중 어느 것이 ...,3
5,현재 전 세계적으로 내재 변동성이 거의 평탄한 지수나 주식이 존재하는지에 대한 예시...,1
6,유럽식 금융 옵션에 대한 질문입니다. 현재 ITM 상태의 콜옵션에 대해 만기가 30...,4
7,"율리우스는 최근 변동금리 채권의 가격 책정 요소에 대해 연구하고 있는데, 그는 리보...",3
8,EBIT을 계산할 때 이자 수익이 포함되어야 하는지 여부에 대한 올바른 이해를 위해...,1
9,편리성 수익이 있는 자산의 경제적 가치를 평가할 때 가장 적절한 접근 방식은 무엇인...,2


In [130]:
import openai
from dotenv import load_dotenv
import os

dotenv_path = os.path.join(os.getcwd(), 'KRX', '.env')
load_dotenv(dotenv_path)

openai_key = os.getenv('OPENAI_TEAM_API_KEY')
client = openai.OpenAI(api_key=openai_key)

# 중복 정답 유무 체크

In [131]:
system_prompt ="""
You are a financial expert.
Read the following questions and choose the answer.
It could be multiple correct answers.
Just write a number or numbers.
"""

user_prompt="""
Question : {}
"""

In [132]:
from tqdm import tqdm

gpt_answer = []
for _, row_item in tqdm(df.iterrows(), total=len(df)):
    answer = int(row_item['answer'])
    question = str(row_item['question'])

    system_msg = {'role': 'system', 'content':system_prompt}
    user_msg = {'role':'user', 'content':user_prompt.format(question)}

    response = client.chat.completions.create(
        model = 'gpt-4o-mini',
        messages = [system_msg, user_msg]
    )

    response_text = response.choices[0].message.content

    gpt_4o_answer = []
    for i in range(6):
        if str(i) in response_text:
            gpt_4o_answer.append(i)

    gpt_answer.append(gpt_4o_answer)

  0%|          | 0/98 [00:00<?, ?it/s]

100%|██████████| 98/98 [01:13<00:00,  1.33it/s]


# 중복 정답 선택지 바꾸기

In [133]:
fixed_system_prompt = """
Your role is to correct the options for financial matters. You will be given the problem and the option number that you will need to rewrite. Please correct the given option number's sentence for your question by meeting the following conditions.
1. The selection must not deviate from the subject of the problem.
2. It should undoubtedly be corrected with a perfect incorrect answer.
3. It must not be modified in the same sense as other fingerprints.
4. The length selected must be similar to the length of other options.
You must only write correct options which satisfy above conditions.
"""

fixed_user_prompt="""
Question: {}
fixed_numbers : {}
"""

In [134]:
fixed_results = []
df['gpt_answer'] = gpt_answer
df

,question,answer,gpt_answer
0,블랙-숄즈(Black-Scholes) 모델을 사용하여 감마(Gamma)를 계산할 때...,3,"[3, 5]"
1,"다음 수식으로 정의된 파생상품의 지급금 가격을 계산하고, 이 가격이 주어진 유럽형 ...",0,"[1, 2, 4]"
2,스팟 가격과 옵션 내재 변동성 간의 상관관계인 $\rho$의 중요성에 대해 설명할 ...,5,"[1, 2, 5]"
3,"GARCH(1,1) 모델을 사용할 때, 어떤 이유로 해당 모델을 선택하는 것이 잘못...",1,"[4, 5]"
4,금융 시장에서 월요일 효과와 1월 효과가 나타나는 원인에 대해 다음 중 어느 것이 ...,3,[3]
...,...,...,...
93,다음 중 가격 커널을 정의할 때 이토의 레마를 적용하는 과정에 대해 가장 적절하지 ...,5,[5]
94,"과거 옵션 데이터의 접근성을 고려할 때, 다음 중 어떤 데이터 제공업체가 가장 적합...",5,"[4, 5]"
95,"코인타그레이션의 개념과 관련하여, 다음의 선택지 중 어떤 것이 코인타그레이션 벡터의...",1,"[1, 2, 3, 4, 5]"
96,중국의 고속철도 투자와 한국의 교육 및 인프라에 대한 정부의 전략적 투자가 경제 성...,3,"[1, 3, 4]"


In [135]:
for idx, row_item in tqdm(df.iterrows(), total=len(df)):
    question = row_item['question']
    answer = row_item['answer']
    gpt_answer = row_item['gpt_answer']
    fixed_question = ''

    need_to_fix_numbers = [num for num in gpt_answer if num != answer]
    if len(need_to_fix_numbers) >= 1:
        fixed_numbers_str = ', '.join(map(str, need_to_fix_numbers))
        fixed_system_msg = {'role':'system', 'content': fixed_system_prompt}
        fixed_user_msg={'role':'user', 'content': fixed_user_prompt.format(question, fixed_numbers_str)}

        response = client.chat.completions.create(
            model='gpt-4o-mini',
            messages = [fixed_system_msg, fixed_user_msg]
        )

        result = response.choices[0].message.content
        question_lines = question.split('\n')
        for fixed_choice in result.split('\n'):
            fixed_choice_text = fixed_choice.strip()
                
            for i in range(1, len(question_lines)):
                if fixed_choice_text.startswith(str(i)):
                    question_lines[i] = fixed_choice_text

        fixed_question = '\n'.join(question_lines)

    if fixed_question:
        new_item = {'question':fixed_question, 'answer':answer}
    else:
        new_item = {'question':question, 'answer':answer}

    fixed_results.append(new_item)
            


  0%|          | 0/98 [00:00<?, ?it/s]

100%|██████████| 98/98 [02:20<00:00,  1.43s/it]


# 데이터 전처리&저장

In [136]:
fixed_df = pd.DataFrame(fixed_results)
fixed_df



,question,answer
0,블랙-숄즈(Black-Scholes) 모델을 사용하여 감마(Gamma)를 계산할 때...,3
1,"다음 수식으로 정의된 파생상품의 지급금 가격을 계산하고, 이 가격이 주어진 유럽형 ...",0
2,스팟 가격과 옵션 내재 변동성 간의 상관관계인 $\rho$의 중요성에 대해 설명할 ...,5
3,"GARCH(1,1) 모델을 사용할 때, 어떤 이유로 해당 모델을 선택하는 것이 잘못...",1
4,금융 시장에서 월요일 효과와 1월 효과가 나타나는 원인에 대해 다음 중 어느 것이 ...,3
...,...,...
93,다음 중 가격 커널을 정의할 때 이토의 레마를 적용하는 과정에 대해 가장 적절하지 ...,5
94,"과거 옵션 데이터의 접근성을 고려할 때, 다음 중 어떤 데이터 제공업체가 가장 적합...",5
95,"코인타그레이션의 개념과 관련하여, 다음의 선택지 중 어떤 것이 코인타그레이션 벡터의...",1
96,중국의 고속철도 투자와 한국의 교육 및 인프라에 대한 정부의 전략적 투자가 경제 성...,3


In [137]:
import re

def preprocess_questions(question):
    question_lines = question.split('\n')

    number_to_eng = {
        '1. ': 'A. ',
        '2. ': 'B. ',
        '3. ': 'C. ',
        '4. ': 'D. ',
        '5. ': 'E. ',
    }

    new_question_lines = []
    for idx, line in enumerate(question_lines):
        new_line = ''
        for i in range(6):
            choice_string = f"{str(i)}. "
            if line.startswith(choice_string):
                new_line = re.sub(choice_string, number_to_eng[choice_string], line)
                break
        if new_line:
            new_question_lines.append(new_line)
        else:
            new_question_lines.append(line)
    
    question_text = "### 질문: " + new_question_lines[0]
    choice_text = "### 선택지:\n" + '\n'.join(new_question_lines[1:])
    result = question_text + "\n" + choice_text

    return result

def preprocess_answer(answer):
    answer = int(answer)
    
    number_to_eng = {
        1: 'A',
        2: 'B',
        3: 'C',
        4: 'D',
        5: 'E',
    }

    if 0 < answer < 6:
        result = number_to_eng[answer] 
    else:
        result = 0
    return result

In [138]:
fixed_df['question'] = fixed_df['question'].apply(preprocess_questions)
fixed_df['answer'] = fixed_df['answer'].apply(preprocess_answer)

In [139]:
def is_numeric(val):
    if val == 0:
        return False
    else:
        return True 

print(len(fixed_df))
fixed_df = fixed_df[fixed_df['answer'].apply(is_numeric)]
fixed_df = fixed_df.reset_index(drop=True)
print(len(fixed_df))

98
91


In [140]:
fixed_df.to_csv('/root/KRX_LLM/data/fixed_krx_sample_mcqa_data4.csv', index=False)